In [12]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import os
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (2.0,2.0)
matplotlib.rcParams['image.cmap'] = 'gray'

__location__ = os.path.realpath(os.getcwd())
filename=os.path.join(__location__, 'blemish.png')
source = cv2.imread(filename,1)
source=cv2.resize(source, None, fx= 0.75, fy= 0.5, interpolation= cv2.INTER_LINEAR)

##### Compare boundary intensity between original and patch #####
def PatchBoundary(original, patch):
    originalBound=original.copy()
    wx,wy, ch=original.shape
    originalBound[2:wx-2, 2:wy-2]=0
    Omean=np.mean(originalBound)
    patchBound=patch.copy()
    patchBound[2:wx-2, 2:wy-2]=0
    Pmean=np.mean(patchBound)
    
    if ((Pmean/Omean)<=1.1) and ((Pmean/Omean)>=0.9):
        return 1
    else: 
        return 0

##### Main algorithm to compute gradient and select patch #####
def RemoveBlemish(action, x, y, flags, userdata):
  # Referencing global variables 
  global columnC, rowC, source, patchR
  # Action to be taken when left mouse button is pressed
  OKtoPatch=0
  GradList=[]
  PatchC=[]
  if action==cv2.EVENT_LBUTTONDOWN:
    if (x-3*patchR)>=0 and (y-3*patchR)>=0 and (x+3*patchR)>=0 and (y+3*patchR)>=0:
        columnC=x
        rowC=y
        targetIm=source[rowC-patchR:rowC+patchR, columnC-patchR:columnC+patchR]
                
        GradList=[]
        PatchC=[]
        
        for i in range(3):
            for j in range(3):
                columngC=x-2*patchR+i*(2*patchR)
                rowgC=y-2*patchR+j*(2*patchR)
                PatchC.append([rowgC,columngC])
                gPatch=source[rowgC-patchR:rowgC+patchR, columngC-patchR:columngC+patchR]
                
        #calculate the Sobel gradient
                sobelx64f = cv2.Sobel(gPatch,cv2.CV_64F,1,0,ksize=5)
                abs_sobel64f = np.absolute(sobelx64f)
                sobel_8u = np.uint8(abs_sobel64f)
                gradx=np.mean(sobel_8u)
                sobely64f = cv2.Sobel(gPatch,cv2.CV_64F,0,1,ksize=5)
                abs_sobel64f = np.absolute(sobely64f)
                sobel_8u = np.uint8(abs_sobel64f)
                grady=np.mean(sobel_8u)
                
                #Check Boundary similarity
                if PatchBoundary(targetIm,gPatch):
                    GradList.append(np.sqrt(gradx**2+grady**2))
        #Find minimum gradient
        index=np.argmin(GradList)

        rowgC=PatchC[index][0]
        columngC=PatchC[index][1]
        newPatch=source[rowgC-patchR:rowgC+patchR, columngC-patchR:columngC+patchR]
        
        
        newCenter=(columnC, rowC)

        mask=255*np.ones(newPatch.shape, newPatch.dtype)
        source=cv2.seamlessClone(newPatch, source, mask, newCenter, cv2.NORMAL_CLONE)
        
       
    # Action to be taken when left mouse button is released
  elif action==cv2.EVENT_LBUTTONUP:
           
    cv2.imshow("Blemish Removal App",source)

##### Main Loop #####
patchR=25
NewPatchC=[]

replacePatch=0
cv2.namedWindow("Blemish Removal App")
# highgui function called when mouse events occur
cv2.setMouseCallback("Blemish Removal App", RemoveBlemish)
k = 0

# loop until escape character is pressed
while k!=27 :

    cv2.imshow("Blemish Removal App", source)
    cv2.putText(source, "Click to remove blemish. [Esc] to exit.", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (250, 150, 0), 1, cv2.LINE_AA)
        
    k = cv2.waitKey(20) & 0xFF
   
   

cv2.destroyAllWindows()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-qjdp5db9\opencv\modules\imgproc\src\deriv.cpp:419: error: (-215:Assertion failed) !_src.empty() in function 'cv::Sobel'
